In [1]:
import numpy as np

In [2]:
def rastrigin(x: np.ndarray) -> float:
    A = 10
    n = len(x)
    return A*n + sum(x[i]**2 - A*np.cos(2*np.pi*x[i]) for i in range(n))

In [8]:
num_dims = 3
bounds = np.array([(-5.12, 5.12) for _ in range(num_dims)])

In [10]:
bounds

array([[-5.12,  5.12],
       [-5.12,  5.12],
       [-5.12,  5.12]])

In [11]:
bounds.clip(-5, 5)

array([[-5.,  5.],
       [-5.,  5.],
       [-5.,  5.]])

In [77]:
class Particle:
    swarm_best_position = None
    swarm_best_value = float('inf')

    def __init__(self, bounds: np.ndarray, f: callable, c_inertia: float, c_social: float, c_cognitive: float):
        self.lb = bounds[:,0]
        self.ub = bounds[:,1]
        self.c_inertia = c_inertia
        self.c_social = c_social
        self.c_cognitive = c_cognitive
        self.f = f

        self.position = np.random.uniform(low=self.lb, high=self.ub, size=len(bounds))
        self.velocity = np.random.uniform(-1, 1, size=len(bounds))
        self.value = f(self.position)

        self.best_position = self.position.copy()
        self.best_value = self.value

        if self.value < Particle.swarm_best_value:
            Particle.swarm_best_value = self.value
            Particle.swarm_best_position = self.position.copy()

    def move(self):
        self.position += self.velocity
        self.position = self.position.clip(self.lb, self.ub)
        self.value = self.f(self.position)

        if self.value < self.best_value:
            self.best_value = self.value
            self.best_position = self.position.copy()
            if self.value < Particle.swarm_best_value:
                Particle.swarm_best_value = self.value
                Particle.swarm_best_position = self.position.copy()


        # self.velocity = inertia + social_velocity + cognitive_velocity
        social_velocity = self.c_social * np.random.random(size=len(self.position)) * (self.swarm_best_position - self.position)
        cognitive_velocity = self.c_cognitive * np.random.random(size=len(self.position)) * (self.best_position - self.position)
        inertia = self.c_inertia * self.velocity

        self.velocity = inertia + cognitive_velocity + social_velocity

In [84]:
def pso(num_particles: int, num_iters: int, bounds: np.ndarray):
    swarm = [Particle(bounds=bounds, f=rastrigin, c_inertia=0.2, c_social=1, c_cognitive=1.5) for _ in range(num_particles)]
    for i in range(num_iters):
        for particle in swarm:
            particle.move()
    
    return Particle.swarm_best_position, Particle.swarm_best_value

In [89]:
Particle.swarm_best_position = None
Particle.swarm_best_value = float('inf')
pso(num_particles=30, num_iters=1000, bounds=bounds)

(array([ 1.34973015e-04, -5.22385138e-07,  7.55805912e-05]),
 np.float64(4.7476044500172065e-06))